In [1]:
# !pip3 install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for numpy>=1.17.0 from https://files.pythonhosted.org/packages/75/5b/ca6c8bd14007e5ca171c7c03102d17b4f4e0ceb53957e8c44343a9546dcc/numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Obtaining dependency information for scipy>=0.16 from https://files.pythonhosted.org/packages/a1/72/8d2b7815d754e52b31ebcacf93111581f6948d96910a1a665b8cefc5cfe1/scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl.metadata
  Using cached scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (60 kB)
  Obtaining dependency information for tqdm>=4.27 from https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558ab34/tqdm-4.66.2-py3-none-any.

In [2]:
# !wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !unzip ml-latest-small.zip

--2024-04-03 13:36:10--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   840KB/s    in 1.1s    

2024-04-03 13:36:13 (840 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [6]:
import numpy as np
from scipy.sparse import csr_matrix

with open("ml-latest-small/ratings.csv", "r") as f:
    print(f.readline())

    rows = []
    cols = []
    data = []

    for line in f:
        uid, mid, _, _ = line.split(",")
        rows.append(int(uid))
        cols.append(int(mid))
        data.append(1)

rows = np.array(rows)
cols = np.array(cols)
data = np.array(data)

### make the csr data
ratings_csr = csr_matrix( (data, (rows, cols)) )

userId,movieId,rating,timestamp



In [9]:
from implicit import bpr

# factors default = 100
# 이로인해 오래걸리기 때문에 현재는 10으로 설정
model = bpr.BayesianPersonalizedRanking(factors = 10)
model.fit(ratings_csr)

# train_auc -> train data set에서 잘 맞추는 정도?

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:01<00:00, 51.55it/s, train_auc=90.30%, skipped=24.87%]


In [10]:
users = [2, 3]
ids, scores = model.recommend(users, ratings_csr[users])
print(ids) # 유저에게 추천하는 아이템의 이름
print(scores) # 내부적으로 정해진 추천 점수

[[122904 134130 112556 112852  84152 139385 109374  80463  69122  89745]
 [  3061   2352   1414   1269   1238   4713   2971   3404   1019   2901]]
[[3.8175251 3.6596115 3.601283  3.534018  3.4999392 3.4010913 3.3381462
  3.3298419 3.3270571 3.3151565]
 [2.2930713 2.262011  2.2375998 2.2236404 2.2204537 2.2015321 2.2008445
  2.1898289 2.1729798 2.1535513]]


In [13]:
print(model.user_factors)
# user 수만큼 벡터가 존재한다.
model.user_factors.shape
# factors의 마지막 값을 바이오스로 활용한다.

[[ 0.          0.          0.         ...  0.          0.
   1.        ]
 [ 0.12280127 -0.8148106  -0.3037428  ... -1.3468064   1.4359229
   1.        ]
 [-0.89448166 -0.04790242  0.30201313 ...  0.816144   -0.44200027
   1.        ]
 ...
 [-0.54553807  1.1838983  -0.32357582 ... -0.60850435  2.4432578
   1.        ]
 [ 0.18648137 -0.16681994 -0.32527804 ... -0.35707948  0.02169952
   1.        ]
 [-2.164012   -0.15486223 -0.2135257  ...  0.722474   -0.9217359
   1.        ]]


(611, 11)

In [14]:
# !pip3 install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/4a/0e/e4e033371a7cba9da0db5ccb507a9174e41b9c29189a932d01f2f61ecfc0/torch-2.2.2-cp312-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/8b/69/acdf492db27dea7be5c63053230130e0574fd8a376de3555d5f8bbc3d3ad/filelock-3.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/d2/05/e6600db80270777c4a64238a98d442f0fd07cc8915be2a1c16da7f2b9e74/sympy-1.12-py3-none-any.whl.metadata
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency informat

In [16]:
# pytorch를 이용하여 fit을 직접 구현하기

import torch

users = torch.from_numpy(rows)
items = torch.from_numpy(cols)

# 각 요소의 갯수를 지정한다.
n_factors = 10
n_items = max(items) + 1
n_users = max(users) + 1

# 각 아이템마다 10차원짜리 factor를 생성한다.
item_bias = torch.randn(n_items, requires_grad=True)
item_factor = torch.randn(n_items, n_factors, requires_grad=True)
user_factor = torch.randn(n_users, n_factors, requires_grad=True)

In [19]:
optim = torch.optim.Adam([item_bias, item_factor, user_factor], lr=0.1)
logsigmoid = torch.nn.LogSigmoid()
lmd = 0.01

for epoch in range(50):

    neg_items = torch.randint(1, n_items, (len(items), ))

    pos_pref = item_bias[items] + (user_factor[users] * item_factor[items]).sum(dim=1)
    neg_pref = item_bias[neg_items] + (user_factor[users] * item_factor[neg_items]).sum(dim=1)
    reg = (item_bias ** 2).sum() + (user_factor ** 2).sum() + (item_factor ** 2).sum()

    # object_function = logsigmoid(pos_pref - neg_pref).sum() - lmd * reg
    cost = -logsigmoid(pos_pref - neg_pref).sum() + lmd * reg

    optim.zero_grad()
    cost.backward()
    optim.step()

    with torch.no_grad():
        # 맞춘 애들의 개수 / 
        train_acc = (pos_pref > neg_pref).sum() / len(pos_pref)
        print(f"epoch: {epoch}, train acc: {train_acc}")

epoch: 0, train acc: 0.8614383935928345
epoch: 1, train acc: 0.9040421843528748
epoch: 2, train acc: 0.9384446144104004
epoch: 3, train acc: 0.9613332748413086
epoch: 4, train acc: 0.9746122360229492
epoch: 5, train acc: 0.9830517172813416
epoch: 6, train acc: 0.9877027869224548
epoch: 7, train acc: 0.9902812242507935
epoch: 8, train acc: 0.9916993975639343
epoch: 9, train acc: 0.9924332499504089
epoch: 10, train acc: 0.9931274652481079
epoch: 11, train acc: 0.9933555722236633
epoch: 12, train acc: 0.9933158755302429
epoch: 13, train acc: 0.9938910603523254
epoch: 14, train acc: 0.9937323927879333
epoch: 15, train acc: 0.9934646487236023
epoch: 16, train acc: 0.993365466594696
epoch: 17, train acc: 0.9937224984169006
epoch: 18, train acc: 0.9938216805458069
epoch: 19, train acc: 0.9939208030700684
epoch: 20, train acc: 0.9933357238769531
epoch: 21, train acc: 0.9938712120056152
epoch: 22, train acc: 0.9934844970703125
epoch: 23, train acc: 0.9937819838523865
epoch: 24, train acc: 0.994

In [20]:
with torch.no_grad():
   
    n_outputs = 10
    uid = 2
    
    scores_all = item_bias + (user_factor[uid] * item_factor).sum(dim=1) 
    scores_all = scores_all.numpy()
    
    ids = np.argsort(scores_all)[::-1]
    pure_ids = ids[np.isin(ids, items[users == uid], invert=True)]
    
    pure_ids = pure_ids[:n_outputs]
    scores = scores_all[pure_ids]
   
    print(pure_ids)
    print(scores)

[139385 122918 122920   1375 159093  79592  55765   4971   4963   4262]
[16.516188  15.695243  15.158437  14.920969  14.626219  14.591423
 14.513702  14.4249935 14.378375  14.219963 ]
